In [ ]:
# необходимые пакеты

# ! pip install ipywidgets
# ! pip install IPython
# ! pip install ipyfilechooser
# ! pip install wave
# ! pip install pvrecorder
# ! pip install vosk
# ! pip install pydub
# ! pip install torch
# ! pip install transformers
# ! pip install pyaudio

In [ ]:
# UI
import ipywidgets as widgets
from IPython.display import display

# выбор рабочей директории
from ipyfilechooser import FileChooser

# работа с wav-файлами
import wave

# запись звука
from pvrecorder import PvRecorder
import pyaudio

# работа с потоками
import threading

# библиотека транскрибации Vosk и необходимые ей пакеты
from vosk import Model
from vosk import KaldiRecognizer
from pydub import AudioSegment

# закачка файлов и работа с архивами
import urllib
import zipfile

# вспомогательные библиотеки
import struct
from enum import Enum
from datetime import datetime
from pathlib import Path
import json
import os
import functools


In [ ]:
# директория для хранения языковой модели, аудиозаписей и логов
# по умолчанию '[текущая папка]\speech_recognition'
# можно задать явно, к примеру:
# ROOT_WORK_FOLDER = r'd:/test/record_sound'

ROOT_WORK_FOLDER = str(Path(os.getcwd(), 'speech_recognition'))

# инициализация рабочей директории
if not os.path.exists(ROOT_WORK_FOLDER):

    Path(ROOT_WORK_FOLDER).mkdir(parents=True, exist_ok=True)

In [ ]:
# общий класс для логирования
class logger:

    class log_level(Enum):
        INFO    = 'INFO'
        WARNING = 'WARNING'
        ERROR   = 'ERROR'

    def __init__(self, print_output = False, work_folder = ROOT_WORK_FOLDER):

        # помимо записи в файл выводить print
        self.print_output = print_output

        self.file = str(Path(work_folder, 'logs').with_suffix('.txt'))

        if not os.path.exists(work_folder):
            return

        dt = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

        with open(self.file, 'a', encoding = 'utf-8') as f:
            f.write(f'\n ============[{dt}]============')


    def log(self, message, level):

        if not os.path.exists(self.file):
            return

        dt = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

        msg = f'\n [{dt}][{level}]: {message}'
        with open(self.file, 'a', encoding = 'utf-8') as f:
            f.write(msg)

        if self.print_output == True:
            print(msg)


    def info(self, message):
        self.log(message, self.log_level.INFO.value)


    def warn(self, message):
        self.log(message, self.log_level.WARNING.value)


    def error(self, message):
        self.log(message, self.log_level().ERROR.value)

log = logger()

In [ ]:
# скачивание языковой модели
try:

    # модель для распознавания русской речи
    language_model_url = "https://alphacephei.com/vosk/models/vosk-model-ru-0.22.zip"

    language_model_path = str(Path(ROOT_WORK_FOLDER, 'vosk-model-ru-0.22'))

    if not os.path.exists(language_model_path):

        log.info(f"Скачивание языковой модели: {language_model_url}...")

        zip_path, _ = urllib.request.urlretrieve(language_model_url)
        with zipfile.ZipFile(zip_path, "r") as f:
            f.extractall(ROOT_WORK_FOLDER)

        log.info(f"Языковая модель сохранена в: {ROOT_WORK_FOLDER}")

except Exception as e:

    log.error(f'Ошибка при скачивании и распаковке языковой модели: {str(e)}')


# настройки канальности и частоты звукового файла
class sound_settings:
    FRAME_RATE = 16000
    CHANNELS = 1

# состояния классов обработки звука
class worker_state(Enum):
    STOPPED = 0
    RUNNING = 1

In [ ]:
# запись звука
class speech_recorder:

    def __init__(self, log):

        self.logger = log if log is not None else logger(print_output = True)

        self.logger.info('Инициализация класса звукозаписи...')

        self.state = worker_state.STOPPED
        self.__worker_thread = None

        # кнопка записи
        self.__button_widget = widgets.Button(
            disabled = False,
            tooltip = 'Запись звука',
            icon = 'microphone',
            button_style = "primary"
        )

        self.__button_widget.on_click(self.__run_worker_thread)

        # выбор рабочей директории для сохранения записи
        self.__file_chooser = FileChooser(
                                path = ROOT_WORK_FOLDER,
                                select_default = True,
                                select_desc = 'Выбрать',
                                change_desc = 'Изменить',
                                title = 'Директория для сохранения аудиофайла:',
                                show_only_dirs = True
                            )
        self.__file_chooser.layout.width = '100%'
        self.__file_chooser.register_callback(self.__file_chooser_callback)

        self.logger.info('Инициализация завершена')


    # получить виджеты для рендера
    def get_widgets(self):
        return widgets.VBox([self.__file_chooser, self.__button_widget])


    # изменить внешний вид кнопки в зависимости от состояния класса
    def __toggle_button_state(self, button, state):
        button.button_style = state[0]
        button.tooltip = state[1]
        button.icon = state[2]


    # запуск записи в отдельном потоке
    def __run_worker_thread(self, button):

        if(self.__file_chooser.selected_path is None):
            self.logger.warn('Выберите директорию для сохранения аудиофайла')
            return

        if(self.state == worker_state.STOPPED):

            self.state = worker_state.RUNNING

            self.__worker_thread = threading.Thread(target = self.__start_recording)
            self.__worker_thread.start()

            self.__toggle_button_state(self.__button_widget, ['success', 'Остановить', 'microphone-slash'])

        else:
            self.state = worker_state.STOPPED
            self.__worker_thread.join()
            self.__worker_thread = None

            self.__toggle_button_state(self.__button_widget, ['primary', 'Запись звука', 'microphone'])


    # запись аудио
    def __start_recording(self):

        self.logger.info('Начало записи звука')

        try:

            recorder = PvRecorder(device_index = -1, frame_length = 512)
            audio = []

            if self.state == worker_state.RUNNING:
                recorder.start()

            while self.state == worker_state.RUNNING:
                frame = recorder.read()
                audio.extend(frame)

            recorder.stop()

            filename = f'recording_{datetime.now().strftime("%d_%m_%Y_%H_%M_%S")}'
            path = str(Path(self.__file_chooser.selected_path, filename).with_suffix('.wav'))

            self.logger.info(f'Сохранение аудиозаписи: {path}')

            with wave.open(path, 'w') as f:
                f.setparams((sound_settings.CHANNELS, 2, sound_settings.FRAME_RATE, 512, "NONE", "NONE"))
                f.writeframes(struct.pack("h" * len(audio), *audio))

        finally:
            recorder.delete()


    def __file_chooser_callback(self):
        self.logger.info(f'Выбрана рабочая директория для сохранения аудиозаписи: {self.__file_chooser.selected_path}')


In [ ]:
# распознавание звукового файла
class speech_transcriber:


    class transcribe_type(Enum):
        FROM_UPLOAD = 0,
        ON_THE_FLY = 1


    def __init__(self, log, root_folder = ROOT_WORK_FOLDER, language_model_folder = 'vosk-model-ru-0.22'):

        self.logger = log if log is not None else logger(print_output = True)

        self.logger.info('Инициализация класса распознавания речи...')

        # пока только .wav файлы, для остальных не так-то просто правильно установить декодеры :)
        self.supported_files = ['wav']

        self.state = worker_state.STOPPED

        self.transcribed_text = None

        # инициализация папок
        self.root_folder = root_folder
        self.language_model_folder = str(Path(self.root_folder, language_model_folder))

        # инициализация модели распознавания
        self.model = Model(language_model_path)
        self.recognizer = KaldiRecognizer(self.model, sound_settings.FRAME_RATE)
        self.recognizer.SetWords(True)

        # выбор аудиофайла
        self.__file_chooser = FileChooser(
                                path = ROOT_WORK_FOLDER,
                                select_desc = 'Выбрать',
                                change_desc = 'Изменить',
                                title = 'Выбрать аудиофайл для распознавания:',
                                filter_pattern = ['*.wav']
                            )
        self.__file_chooser.layout.width = '100%'
        self.__file_chooser.register_callback(self.__file_chooser_callback)

         # кнопка распознавания загруженного аудиофайла
        self.__button_transcribe_widget = widgets.Button(
            disabled = False,
            tooltip = 'Распознать загруженный аудиофайл',
            icon = 'book',
            button_style = "primary",
            filter_pattern = []
        )

        # кнопка распознавания аудиофайла на лету
        self.__button_transcribe_on_the_fly_widget = widgets.Button(
            disabled = False,
            tooltip = 'Распознать на лету',
            icon = 'microphone',
            button_style = "primary",
            filter_pattern = []
        )

        self.__button_transcribe_on_the_fly_widget.on_click(functools.partial(self.__run_transcribe_worker_thread, type = self.transcribe_type.ON_THE_FLY.value))
        self.__button_transcribe_widget.on_click(functools.partial(self.__run_transcribe_worker_thread, type = self.transcribe_type.FROM_UPLOAD.value))


        # поле для отображения распознанного текста
        self.__transcribe_text_widget = widgets.Textarea(
            value = 'Распознанный текст:',
            placeholder = '',
            description = '',
            disabled = False,
            layout = widgets.Layout(width = 'auto', height = '300px')
        )
        self.__transcribe_text_widget.style.font_size = '20px'

        self.logger.info('Инициализация завершена')


    # получить виджеты для рендера
    def get_widgets(self):
        return widgets.VBox([self.__file_chooser,
                             self.__button_transcribe_widget,
                             self.__button_transcribe_on_the_fly_widget,
                             self.__transcribe_text_widget,
                            ])


    # изменить внешний вид кнопки в зависимости от состояния класса
    def __toggle_button_state(self, button, state):
        button.button_style = state[0]
        button.tooltip = state[1]
        button.icon = state[2]


    # запуск распознавания в отдельном потоке
    def __run_transcribe_worker_thread(self, button, type):

        is_from_upload = type == self.transcribe_type.FROM_UPLOAD.value

        button = None
        method = None
        button_state = None

        if is_from_upload == True:
            button = self.__button_transcribe_widget
            method = self.__start_transcribing
            button_state = [
                ['success', 'Идёт распознование', 'spinner'],
                ['primary', 'Распознать аудиофайл', 'book']
            ]
        else:
            button = self.__button_transcribe_on_the_fly_widget
            method = self.__start_transcribing_on_the_fly
            button_state = [
                ['success', 'Идёт распознование', 'microphone-slash'],
                ['primary', 'Распознать на лету', 'microphone']
            ]

        if(is_from_upload == True and self.__file_chooser.selected_path is None):
            self.logger.warn('Не выбран аудиофайл для распознавания')
            return

        # если запущено распознавание загруженного файла
        # игнорируем последующие нажатия кнопки
        if(self.state == worker_state.STOPPED):

            self.state = worker_state.RUNNING

            self.__worker_thread = threading.Thread(target = method)
            self.__worker_thread.start()

            self.__toggle_button_state(button, button_state[0])

            if is_from_upload == True and self.__worker_thread is not None:

                self.__worker_thread.join()
                self.__worker_thread = None

                self.state = worker_state.STOPPED

                self.__toggle_button_state(button, button_state[1])

        # при распознавании в реальном времени последующее нажатие кнопки остановит поток
        elif is_from_upload == False:

            self.state = worker_state.STOPPED

            if self.__worker_thread is not None:

                self.__worker_thread.join()
                self.__worker_thread = None

            self.__toggle_button_state(button, button_state[1])


    # распознавание загруженного аудиофайла
    def __start_transcribing(self):

        if(self.__file_chooser.selected_path is None):
            self.logger.warn('Выберите .wav аудиофайл')
            return

        is_valid, ext = self.is_valid_file(self.__file_chooser.selected)

        if not is_valid:
            self.logger.error('Недопустимый формат файла')
            return

        self.logger.info('Распознавание аудиофайла...')

        self.recognizer = KaldiRecognizer(self.model, sound_settings.FRAME_RATE)
        self.recognizer.SetWords(True)
        self.__transcribe_text_widget.value = ' '
        self.transcribed_text = None

        data = None

        # wav
        if ext == self.supported_files[0]:
            data = AudioSegment.from_wav(self.__file_chooser.selected)
        else:
            return

        data = data.set_channels(sound_settings.CHANNELS)
        data = data.set_frame_rate(sound_settings.FRAME_RATE)

        # текстовый результат
        self.recognizer.AcceptWaveform(data.raw_data)
        result = self.recognizer.Result()
        self.transcribed_text = json.loads(result)["text"]

        self.logger.info('Аудиофайл распознан')

        self.__transcribe_text_widget.value = self.transcribed_text

        self.__save(self.__file_chooser.selected, self.transcribed_text)



    # распознавание на лету
    def __start_transcribing_on_the_fly(self):

        self.logger.info('Распознование речи в реальном времени')

        self.__transcribe_text_widget.value = ' '
        self.transcribed_text = None

        self.recognizer = KaldiRecognizer(self.model, sound_settings.FRAME_RATE)

        p = pyaudio.PyAudio()
        stream = p.open(
            format = pyaudio.paInt16,
            channels = sound_settings.CHANNELS,
            rate = sound_settings.FRAME_RATE,
            input = True,
            frames_per_buffer = sound_settings.FRAME_RATE
        )
        stream.start_stream()

        result = ''
        while self.state == worker_state.RUNNING:
            data = stream.read(int(sound_settings.FRAME_RATE))

            chunk = self.recognizer.Result() if self.recognizer.AcceptWaveform(data) else self.recognizer.PartialResult()

            partial = ''
            text = ''
            # если есть только частичные результаты - выведем их на экран в реальном времени
            # как только появится полностью распознанная часть текста - добавим её к финальному результату
            json_ = json.loads(chunk)
            if 'text' in json_:
                if len(json_['text']) > 0:
                    text = json_['text']
                    result += f' {text}'
            if 'partial' in json_:
                partial = json_['partial']

            if len(partial) > 0:
                self.__transcribe_text_widget.value = result + f' {partial}'
            if len(text) > 0:
                self.__transcribe_text_widget.value = result

        result += json.loads(self.recognizer.FinalResult())['text']
        self.__transcribe_text_widget.value = result

        filename = f'transcribe_on_the_fly_{datetime.now().strftime("%d_%m_%Y_%H_%M_%S")}'
        path = str(Path(ROOT_WORK_FOLDER, filename).with_suffix('.txt'))

        self.logger.info('Сохранение распознанной на лету транскрипции:')
        self.__save(path, result)


    # сохранение распознанного текста в рабочую директорию
    def __save(self, file, text):

        file_name = Path(file).stem
        save_path = str(Path(self.root_folder, file_name).with_suffix('.txt'))

        self.logger.info(f"Сохранение распознанного файла: {save_path}")

        with open(save_path, 'w', encoding = 'utf-8') as f:
            f.write(text)


    def __file_chooser_callback(self):
        self.logger.info(f'Выбран файл для распознавания: {self.__file_chooser.selected}')


    # проверка расширения файла
    def is_valid_file(self, filepath):

        is_valid = False

        ext = Path(filepath).suffix
        ext = ext.replace('.','')
        if ext in self.supported_files:
            is_valid = True

        return is_valid, ext



In [ ]:
# рендер виджетов

sr = speech_recorder(log)
st = speech_transcriber(log) # инициализируется минуту и более, в зависимости от железа

log.print_output = True

header_record = widgets.HTML(value = 'Запись:')
header_transcribe = widgets.HTML(value = 'Распознавание:')

box_record = sr.get_widgets()
box_transcribe = st.get_widgets()

header_record.style.font_size, header_transcribe.style.font_size = '20px', '20px'
header_record.layout.padding, header_transcribe.layout.padding = '25px 0 0 0','50px 0 0 0'
box_record.layout.padding, box_transcribe.layout.padding = '10px 0 10px 0','10px 0 10px 0'

controls = widgets.GridBox(children = [header_record, box_record, header_transcribe, box_transcribe],
        layout=widgets.Layout(
            grid_template_rows = 'auto auto auto auto',
            grid_template_areas='''
            "header_record"
            "box_record"
            "header_transcribe"
            "box_transcribe"
            ''')
    )

display(controls)

